<a href="https://colab.research.google.com/github/beyzanurbayir/hackathon/blob/main/HackathonHepsiBurada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adım 1: Veri Yükleme ve Keşifsel Veri Analizi (EDA)**
Bu projenin ilk aşamasında, bize sağlanan eğitim (train.csv) ve test (test.csv) veri setlerini yükleyip temel bir analizden geçirerek veriyi anlamaya çalıştık. Keşifsel Veri Analizi (EDA), modelleme aşamasına geçmeden önce verinin yapısını, potansiyel sorunlarını ve özelliklerini anlamak için kritik bir adımdır.

In [1]:
#@title 1.1 Gerekli Kütüphanelerin Yüklenmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Grafikler için stil belirleyelim
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

print("Kütüphaneler başarıyla yüklendi.")

Kütüphaneler başarıyla yüklendi.


In [2]:
#@title 1.2 Veri Setlerini Yükleme
try:
    BASE_PATH = '/content'
    train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
    test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
    print("Eğitim ve test veri setleri başarıyla yüklendi.")
    print(f"Eğitim verisi boyutu: {train_df.shape}")
    print(f"Test verisi boyutu: {test_df.shape}")
except FileNotFoundError as e:
    print(f"Hata: Dosya bulunamadı! Lütfen sol taraftaki dosya panelinden 'HepsiburadaHackathon' klasörünün")
    print("ve içindeki 'train.csv' ve 'test.csv' dosyalarının doğru yüklendiğinden emin olun.")
    print(f"Aranan yol: {BASE_PATH}")


Eğitim ve test veri setleri başarıyla yüklendi.
Eğitim verisi boyutu: (848237, 2)
Test verisi boyutu: (217241, 2)


In [ ]:
#@title 1.3 Veriye İlk Bakış (Initial Inspection)
print("\n--- Eğitim Veri Seti (İlk 5 Satır) ---")
# display() fonksiyonu Colab'de tabloları daha güzel gösterir.
display(train_df.head())

print("\n--- Test Veri Seti (İlk 5 Satır) ---")
display(test_df.head())


In [ ]:
#@title 1.4 Veri Seti Bilgileri ve Eksik Değer Kontrolü
print("\n--- Eğitim Veri Seti Bilgileri ---")
train_df.info()

print("\n--- Test Veri Seti Bilgileri ---")
test_df.info()

# Eksik değerleri kontrol edelim
print("\n--- Veri Setlerindeki Eksik Değer Sayıları ---")
print("Eğitim seti eksik değerler:\n", train_df.isnull().sum())
print("\nTest seti eksik değerler:\n", test_df.isnull().sum())

In [ ]:
#@title 1.5 Benzersiz Etiket (Label) Sayısı
unique_labels = train_df['label'].nunique()
print(f"\nEğitim setindeki toplam benzersiz etiket (sınıf) sayısı: {unique_labels}")

In [ ]:
#@title 1.6 Adres Uzunluklarının Analizi ve Görselleştirilmesi
train_df['address_length'] = train_df['address'].str.len()
test_df['address_length'] = test_df['address'].str.len()

plt.figure(figsize=(14, 7))
sns.histplot(train_df['address_length'], color='dodgerblue', label='Eğitim Seti', kde=True, bins=60)
sns.histplot(test_df['address_length'], color='palevioletred', label='Test Seti', kde=True, bins=60)
plt.title('Adres Uzunluklarının Dağılımı (Karakter Sayısı)')
plt.xlabel('Adres Uzunluğu')
plt.ylabel('Frekans')
plt.legend()
plt.show()

print("\n--- Adres Uzunluğu İstatistikleri (Eğitim Seti) ---")
print(train_df['address_length'].describe())

***SONUÇ***

**->Veri Bütünlüğü Mükemmel:** info() ve isnull().sum() çıktıları, hem eğitim hem de test setinde hiç eksik veri olmadığını net bir şekilde doğruluyor. Bu, projenin en başında büyük bir avantajdır çünkü eksik verileri doldurmak için karmaşık stratejiler geliştirmemize gerek kalmıyor. Verimiz temiz ve tam.

**->Problemin Kapsamı ve Zorluğu:** Eğitim setinde 10,390 benzersiz label (sınıf) olması, bu projenin standart bir sınıflandırma probleminden çok daha zorlu olduğunu gösteriyor. Bu durum, modelin on binden fazla farklı adres grubunu ayırt edebilme yeteneğine sahip olması gerektiği anlamına gelir. Bu nedenle, özellik mühendisliği ve model seçimi kritik olacaktır.

**->Adres Metinlerinin Yapısı:** head() çıktılarında gördüğümüz gibi, adresler standart bir formata sahip değil. Büyük/küçük harf kullanımı, kısaltmalar (Mah., Sok., No.), noktalama işaretleri ve yazım tarzları adresten adrese büyük farklılıklar gösteriyor. Bu durum, bir sonraki adım olan Veri Ön İşleme'nin ne kadar önemli olduğunu kanıtlıyor. Metinleri standart bir formata getirmeden modelin anlamlı örüntüler öğrenmesi çok zor olacaktır.

**->Adres Uzunlukları ve Tutarlılık:** address_length istatistikleri, adreslerin ortalama 65 karakter uzunluğunda olduğunu fakat 1 karakterden 250 karaktere kadar geniş bir yelpazede dağıldığını gösteriyor. Özellikle min değerinin 1 olması, veri setinde anlamsız veya hatalı olabilecek birkaç kayıt olabileceğine işaret ediyor.

Daha da önemlisi, oluşturduğun histogram grafiği, eğitim ve test setindeki adres uzunluk dağılımlarının neredeyse birebir aynı olduğunu gösteriyor. Bu, projenin en olumlu bulgularından biridir. Bu tutarlılık, eğitim setinde öğrendiğimiz özelliklerin ve modelin, test setinde de benzer bir veri yapısıyla karşılaşacağını ve dolayısıyla daha başarılı bir genelleme yapabileceğini gösterir.

# **Adım 2: Veri Temizleme ve Ön İşleme (Data Preprocessing)**
Bu adımın amacı, adres metinlerindeki "gürültüyü" ortadan kaldırmak ve tutarlı bir yapı oluşturmaktır. Yapacağımız işlemler şunlardır:

* **Küçük Harfe Çevirme:** Tüm metni küçük harfe dönüştürerek MAH ile mah gibi ifadelerin aynı anlama gelmesini sağlayacağız.

* **Kısaltmaları Genişletme:** mah, sok, no, apt gibi sık kullanılan kısaltmaları tam ve tutarlı karşılıklarıyla (mahallesi, sokak, numara, apartmanı) değiştireceğiz.

* **Noktalama ve Özel Karakterleri Kaldırma:** Adreslerin anlamını taşımayan nokta, virgül, slash (/), (") gibi karakterleri temizleyeceğiz. Sadece harfleri, sayıları ve boşlukları koruyacağız.

* **Gereksiz Boşlukları Temizleme:** Kelimeler arasındaki çoklu boşlukları tek boşluğa indirecek ve metnin başındaki/sonundaki boşlukları kaldıracağız.

In [ ]:
#@title 2.1 Temizleme ve ön işleme fonksiyonunu tanımlayalım
import re # Metin işlemleri için Regular Expression kütüphanesi

# 'train_df' ve 'test_df'nin yüklü olduğunu varsayıyoruz.
# Üzerinde çalışmak için tekrar kopyalarını oluşturalım.
try:
    train_processed_df = train_df.copy()
    test_processed_df = test_df.copy()
    print("İşlem için DataFrame kopyaları başarıyla oluşturuldu.")
except NameError:
    print("Hata: 'train_df' veya 'test_df' bulunamadı. Lütfen 1. Adım'daki veri yükleme kodunu tekrar çalıştırdığınızdan emin olun.")


# 1. Adım: İyileştirilmiş temizleme ve ön işleme fonksiyonu
def preprocess_address(address_text):
    """
    Bu fonksiyon bir adres metnini alır ve standart bir formata getirir.
    - Küçük harfe çevirir.
    - Kısaltmaları genişletir.
    - Noktalama işaretlerini boşluk ile değiştirir. (GÜNCELLENDİ)
    - Fazla boşlukları temizler.
    """
    address_text = str(address_text)
    processed_text = address_text.lower()

    abbreviations = {
        r'\bmah\b': 'mahallesi', r'\bmh\b': 'mahallesi',
        r'\bsok\b': 'sokak', r'\bsk\b': 'sokak',
        r'\bcad\b': 'caddesi', r'\bcd\b': 'caddesi',
        r'\bno\b': 'numara', r'\bapt\b': 'apartmanı',
        r'\bd\b': 'daire', r'\bkat\b': 'kat',
        r'\bblv\b': 'bulvarı', r'\bsit\b': 'sitesi'
    }

    for pattern, replacement in abbreviations.items():
        processed_text = re.sub(pattern, replacement, processed_text)

    # ---> GÜNCELLENEN KISIM <---
    # Noktalama ve özel karakterleri silmek yerine 'boşluk' ile değiştiriyoruz.
    # Bu, 'no:15' gibi ifadelerin 'numara 15' olmasını sağlar.
    processed_text = re.sub(r'[^a-zçğıöşü0-9\s]', ' ', processed_text)

    # Gereksiz boşlukları tek boşluğa indirgeme ve kırpma
    # Bu adım, yukarıdaki işlemden kaynaklanabilecek çift boşlukları da temizler.
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()

    return processed_text

In [ ]:
#@title 2.2 Adım: Fonksiyonu hem eğitim hem de test verisindeki 'address' sütununa uygulayalım
print("\nÖn işleme süreci başlatılıyor...")
train_processed_df['address_processed'] = train_processed_df['address'].apply(preprocess_address)
test_processed_df['address_processed'] = test_processed_df['address'].apply(preprocess_address)
print("Ön işleme tamamlandı.")


# 3. Adım: Yeni sonuçları karşılaştıralım
print("\n--- GÜNCELLENMİŞ ÖRNEKLERİN ÖNCE VE SONRAKİ HALLERİ ---")

# Daha önce baktığımız karmaşık test örneğini tekrar inceleyelim
original_example = test_df.loc[1, 'address']
processed_example = test_processed_df.loc[1, 'address_processed']
print(f"ORİJİNAL : {original_example.replace('/n', ' ')}")
print(f"İŞLENMİŞ : {processed_example}")
print("-" * 20)

# Eğitim setinden de bir örnek
original_example_train = train_df.loc[345, 'address'] # Herhangi bir örnek
processed_example_train = train_processed_df.loc[345, 'address_processed']
print(f"ORİJİNAL : {original_example_train}")
print(f"İŞLENMİŞ : {processed_example_train}")

***SONUÇ***

Keşifsel Veri Analizi (EDA) aşamasında tespit edilen metin verisindeki ham ve düzensiz yapı, bu adımda modelin anlayabileceği standart ve temiz bir formata dönüştürülmüştür. Bu süreç, modelin doğruluğu ve verimliliği için kritik öneme sahiptir.

**->Metin Standardizasyonu ve Tutarlılık:**Adres metinlerindeki tüm harfler, Türkçe karakterler korunarak küçük harfe çevrilmiştir. Mah., sok., no gibi sıkça karşılaşılan kısaltmalar mahallesi, sokak, numara gibi tam ve tutarlı karşılıklarıyla değiştirilmiştir. Bu işlem, aynı anlama gelen farklı ifadeleri tek bir formata indirgeyerek modelin kelime dağarcığını (vocabulary) optimize eder ve öğrenme sürecini kolaylaştırır.

**->Gürültü Giderme ve Yapısal Bütünlüğün Korunması:** Adreslerde anlamsal bir değer taşımayan noktalama işaretleri (., :, / vb.) ve özel karakterler metinden arındırılmıştır. Bu süreçte yapılan en önemli iyileştirme, bu karakterlerin doğrudan silinmesi yerine bir boşluk karakteri ile değiştirilmesi olmuştur. Bu sayede no:15 gibi ifadelerin numara15 şeklinde birleşmesi sorunu önlenmiş, numara 15 gibi kelime ve sayıların doğal yapısı korunmuştur. Bu, modelin kelimeler arasındaki ilişkileri doğru bir şekilde öğrenmesi için hayati bir adımdır.

**->Model İçin Optimum Formata Ulaşma:** Yapılan temizlik işlemleri sonucunda, tüm adresler artık gereksiz boşluklardan arındırılmış, standartlaştırılmış ve kelime bütünlüğü korunmuş bir yapıya kavuşmuştur. Veri seti, bir sonraki aşama olan Özellik Çıkarımı (Feature Engineering) için hazır hale getirilmiştir. Temizlenmiş bu metinler, makine öğrenmesi modellerinin işleyebileceği sayısal vektörlere daha anlamlı bir şekilde dönüştürülebilir.

# **Adım 3: Özellik Çıkarımı (Feature Engineering)**
Bu aşamanın temel amacı, önceki adımda temizlediğimiz metin halindeki adresleri, makine öğrenmesi algoritmalarının anlayabileceği sayısal bir formata, yani vektörlere dönüştürmektir.

Bunun için popüler ve güçlü bir metin temsil yöntemi olan TF-IDF (Term Frequency-Inverse Document Frequency) kullanacağız.

TF-IDF'in Mantığı Nedir?

* **TF (Term Frequency - Terim Sıklığı):** Bir kelimenin bir adreste ne kadar sık geçtiğini ölçer. Bir kelime bir adreste ne kadar çok geçiyorsa, o adres için o kadar önemlidir.

* **IDF (Inverse Document Frequency - Ters Belge Sıklığı):** Bir kelimenin tüm adresler (corpus) genelinde ne kadar nadir olduğunu ölçer. "mahallesi", "sokak" gibi her adreste geçen kelimelerin önemini azaltırken; "çiğdem", "gelincik" gibi daha nadir ve ayırt edici kelimelerin önemini artırır.

Bu iki değerin çarpımı, bir kelimenin belirli bir adres için ne kadar "karakteristik" veya "ayırt edici" olduğunu gösteren bir skor verir.

In [ ]:
#@title 3.1 Adım: TF-IDF Vectorizer'ı Yapılandırma

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Bu ayarlar başlangıç için iyi bir noktadır ve daha sonra optimize edilebilir.
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),  # Hem tekil kelimeleri ('mahallesi') hem de ikili kelime gruplarını ('atatürk mahallesi') yakalar.
    max_features=50000,  # En sık geçen 50,000 kelime/kelime grubunu özellik olarak kullan. Bu, belleği verimli kullanmamızı sağlar.
    min_df=2             # Bir kelimenin özellik olarak sayılması için en az 2 adreste geçmesi gerekir. Bu, yazım hatalarını ve gürültüyü azaltır.
)


In [ ]:
#@title 3.2 Adım: Vectorizer'ı Eğitim Verisi Üzerinde Eğitme ve Verileri Dönüştürme
print("TF-IDF Vectorizer eğitim verisi üzerinde eğitiliyor...")
# fit_transform: Önce kelime dağarcığını öğrenir (fit), sonra metinleri vektörlere dönüştürür (transform).
X = tfidf_vectorizer.fit_transform(train_processed_df['address_processed'])

print("Eğitim verisi TF-IDF vektörlerine dönüştürüldü.")

# Sadece 'transform' kullanarak test verisini dönüştürüyoruz.
# Asla test verisi üzerinde 'fit' yapmayız, çünkü bu veri sızıntısına (data leakage) yol açar.
X_test_final = tfidf_vectorizer.transform(test_processed_df['address_processed'])
print("Test verisi TF-IDF vektörlerine dönüştürüldü.")

# Hedef değişkenimizi (etiketler) bir değişkene atayalım.
y = train_processed_df['label']

# Oluşturulan matrislerin boyutlarını kontrol edelim
print(f"\nEğitim verisi (X) matris boyutu: {X.shape}")
print(f"Hedef değişken (y) vektör boyutu: {y.shape}")
print(f"Final test verisi (X_test_final) matris boyutu: {X_test_final.shape}")
# X.shape çıktısındaki ikinci değer (50000), max_features ile belirlediğimiz özellik sayısını gösterir.


In [ ]:
#@title 3.3 Adım: Eğitim ve Validasyon Setlerine Ayırma
# Veri setimizi model eğitimi ve model performansını doğrulamak (validasyon) için ayırıyoruz.
# stratify=y parametresi, ayırma işlemi sırasında orijinal veri setindeki sınıf oranlarının
# hem eğitim hem de validasyon setlerinde korunmasını sağlar. Bu, çok sınıflı problemlerde çok önemlidir.
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2, # Verinin %20'sini validasyon için ayır
    random_state=42, # Her seferinde aynı ayırma işleminin yapılması için
    stratify=y
)

print("\nVeri, eğitim ve validasyon setlerine ayrıldı:")
print(f"X_train boyutu: {X_train.shape}")
print(f"X_val boyutu: {X_val.shape}")
print(f"y_train boyutu: {y_train.shape}")
print(f"y_val boyutu: {y_val.shape}")

***SONUÇ***

Bu aşamada, modelimizin anlayamadığı ham metin verisini, artık işleyebileceği yapısal ve sayısal bir formata dönüştürdük. Çıktılar bu dönüşümün başarısını ve boyutlarını net bir şekilde ortaya koyuyor.

**->Metinden Sayısal Temsile Geçiş:** Eğitim verisi (X) matris boyutu: (848237, 50000) çıktısı bu adımın en önemli sonucudur. Bu, 848,237 adet adres metninin her birinin artık 50,000 sayıdan oluşan bir vektör ile temsil edildiği anlamına gelir.

Her bir sayı (sütun/özellik), TfidfVectorizer tarafından en önemli olarak belirlenen bir kelimenin veya iki kelimelik bir ifadenin (ngram_range=(1, 2)) o adres için hesaplanmış TF-IDF skorunu ifade eder.

**->Boyut ve Karmaşıklık Kontrolü:** Adres metinlerindeki on binlerce farklı kelime yerine, modelimizin odaklanacağı en önemli 50,000 özelliği max_features parametresi ile bilinçli olarak seçtik. Bu, hem hesaplama süresini ve bellek kullanımını makul bir seviyede tutar hem de çok nadir ve muhtemelen gürültü içeren kelimeleri eleyerek modelin daha iyi genelleme yapmasına yardımcı olur.

**->Veri Setleri Arası Tutarlılık:** Hem eğitim (X) hem de test (X_test_final) matrislerinin ikinci boyutunun (sütun sayısı) aynı (50,000) olduğuna dikkat edin. Bu çok önemlidir. Modelimiz, eğitim verisindeki 50,000 özellikten örüntüler öğrenecek ve tahmin yapacağı test verisinde de tam olarak aynı 50,000 özelliği görmeyi bekleyecektir. Bu tutarlılığı sağlayarak modelin doğru çalışmasını garanti altına aldık.

**->Model Eğitimi ve Değerlendirmesi İçin Hazırlık:** Son olarak, train_test_split ile veriyi eğitim ve validasyon setlerine ayırdık. X_train ve y_train ile modelimizi eğiteceğiz. Ardından, modelin daha önce hiç görmediği X_val verisi üzerindeki performansını y_val etiketleriyle karşılaştırarak modelimizin ne kadar başarılı olduğunu tarafsız bir şekilde ölçeceğiz.

**Özetle:** Soyut bir kavram olan "adres metnini", bir makine öğrenmesi modelinin üzerinde matematiksel işlemler yapabileceği devasa bir sayısal matrise başarıyla dönüştürdük. Artık projenin en can alıcı kısmı olan modelleme aşamasına geçmek için her şey hazır.

# **ADIM 4: Model Geliştirme ve Eğitim**

Bu ilk denememiz için hem hızı hem de TF-IDF gibi seyreltik (sparse) matrislerle iyi çalışma yeteneği nedeniyle Logistic Regression modelini kullanacağız. Bu model, sınıflandırma problemleri için güçlü ve yorumlanması kolay bir başlangıç noktasıdır.

Yapacağımız İşlemler:

1. Scikit-learn kütüphanesinden LogisticRegression modelini çağıracağız.


2. Modeli, bir önceki adımda oluşturduğumuz eğitim verisi (X_train, y_train) ile eğiteceğiz.

3. Eğitilen modelin performansını, daha önce hiç görmediği validasyon verisi (X_val) üzerinde test edeceğiz.

4. Performansı ölçmek için yarışmanın ana değerlendirme metriği olan F1 Skoru'nu ve genel bir metrik olan Doğruluk (Accuracy)'u kullanacağız.

In [ ]:
#@title 4.1 Adım: Modelin Seçimi ve Yapılandırılması
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
import time
import numpy as np

# Logistic Regression, bu tür metin sınıflandırma görevleri için hızlı ve güçlü bir başlangıç modelidir.
model = LogisticRegression(
    solver='saga',      # Büyük veri setleri için optimize edilmiş bir çözücü.
    penalty='l2',       # Standart bir regülarizasyon türü.
    C=1.0,              # Regülarizasyon gücü (varsayılan değer).
    random_state=42,
    n_jobs=-1           # Mevcut tüm CPU çekirdeklerini kullanarak eğitimi hızlandır.
)

In [ ]:
#@title 4.2 Adım: Modelin Eğitilmesi
# Bu işlem, veri setinin büyüklüğüne bağlı olarak birkaç dakika sürebilir.
print("Model eğitimi başlatılıyor...")
start_time = time.time()

# Modeli eğitim verileriyle eğitiyoruz.
model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Model eğitimi {training_time:.2f} saniyede tamamlandı.")



In [ ]:
#@title 4.3 Adım: Validasyon Seti Üzerinde Tahmin Yapma
print("\nValidasyon seti üzerinde tahminler yapılıyor...")
y_pred_val = model.predict(X_val)
print("Tahminler yapıldı.")


In [ ]:
#@title 4.4 Adım: Model Performansının Değerlendirilmesi
print("\n--- Model Performans Değerlendirmesi ---")

# Doğruluk (Accuracy) Skoru: Toplam tahminlerin ne kadarının doğru olduğunu gösterir.
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Doğruluk (Accuracy): {accuracy:.4f}")

# F1 Skoru (Ağırlıklı): Yarışmanın ana metriği budur.
# 'weighted' ortalama, her sınıfın eleman sayısını dikkate alarak F1 skorunu hesaplar.
# Bu, dengesiz sınıf dağılımına sahip veri setleri için daha adil ve anlamlı bir metriktir.
f1 = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ağırlıklı F1 Skoru (Weighted F1 Score): {f1:.4f}")

# Sınıflandırma Raporu (Classification Report)
# Bu rapor, her bir sınıf için Precision, Recall ve F1-score değerlerini detaylı olarak gösterir.
# Ancak 10,000'den fazla sınıf olduğu için raporun tamamını yazdırmak pratik değildir.
# Bu yüzden sadece birkaç sınıf için raporun bir kısmını örnek olarak gösterelim.
print("\nSınıflandırma Raporu (Örnek Sınıflar İçin):")
try:
    # Hem validasyon setinde hem de tahminlerde bulunan ortak etiketleri alalım
    unique_labels_in_report = np.unique(np.concatenate((y_val, y_pred_val)))
    # Bu etiketlerden sadece ilk 10 tanesini raporda gösterelim
    target_labels_to_show = unique_labels_in_report[:10]

    print(classification_report(y_val, y_pred_val, labels=target_labels_to_show, zero_division=0))
except Exception as e:
    print(f"Sınıflandırma raporu oluşturulurken bir hata oluştu: {e}")
    print("Bu durum, validasyon setindeki bazı sınıfların tahminlerde hiç yer almamasından kaynaklanabilir.")